<a href="https://colab.research.google.com/github/Saatvik-Aggarwal/phased-array/blob/main/Beamforming_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
from scipy.constants import c, pi
from scipy import signal

In [3]:
# Setup of constants

carrier_frequency = 10e9  # RF Center frequency (Hz)
lam = c / carrier_frequency  # Wavelength (m)
k = 2 * pi / lam  # wave number

# Define Array Layout
# Uniform Rectangular Array with half-wavelength spacing

d_x = d_y = lam / 2  # element spacing (m) // to avoid grating lobes for moderate scan
N , M = 4 , 4  # ULA 4x4

In [6]:
# Creating a Taylor weighting window
# --- Instead of every element radiation equally, this window smoothly lowers the amplitude
#       toward the edges of the array to reduce sidelobes in the radiation pattern


# Number of nearly constant-level sidelobes in the Taylor distribution
#---Controls how smooth tappering is (larger nbar spreads the transition more evenly)
nbar_n = 3
nbar_m = 3

# Sidelobe Level
sll_n = 25
sll_m = 25

# Taylor Amplitude Weighting
weight_taylor_n = signal.windows.taylor(N, nbar=nbar_n, sll=sll_n)
weight_taylor_m = signal.windows.taylor(M, nbar=nbar_m, sll=sll_m)

In [7]:
def compute_phase_angles (u_0 , v_0, number_bits = 4):

  # Select amplitude tapers

  w_m = weight_taylor_m.astype(float)   # length M
  w_n = weight_taylor_n.astype(float)   # length N

  # Create integer grid

  index_m = np.arange(M)[:, None]      # rows x
  index_n = np.arange(N)[None, :]      # columns y

  # Compute the spatial phase each element needs to point the beam towards
  #---To transmit a beam, give each element a complex weight that cancels
  #   that spatial phase so all elements add in phase in that direction.

  phase_shift = k * (index_m * d_y * v_0 + index_n * d_x * u_0)   # Steering phase shift (the actual delay applied to each element)
  steering_weight = (w_m[:, None] * w_n[None, :]) * np.exp(-1j * phase_shift)  # complex TX weight per element (M x N)

  # Phase of each TX weight in degrees
  phase_array_tx = np.angle(steering_weight, deg=True)

  phase_deg = (phase_array_tx + 360) % 360    # [0, 360]
  number_states = 2**number_bits    # 64 states
  phase_states_tx = np.round(phase_deg / 360 * (number_states - 1)).astype(int)   # Convert to phase states

  return phase_states_tx

In [8]:
# Choose steering angles (Beam Direction)

elevation_steering_angle = 20
azimuth_steering_angle   = 30

# Convert to sine space
u_0 = np.sin(np.deg2rad(elevation_steering_angle)) * np.cos(np.deg2rad(azimuth_steering_angle))
v_0 = np.sin(np.deg2rad(elevation_steering_angle)) * np.sin(np.deg2rad(azimuth_steering_angle))

# Compute phase
phase_states_tx = compute_phase_angles(u_0, v_0)

print(f"4x4 TX phase STATES (quantized) for Elevation = {elevation_steering_angle}°, Azimuth = {azimuth_steering_angle}°")
print(phase_states_tx)



4x4 TX phase STATES (quantized) for Elevation = 20°, Azimuth = 30°
[[ 0 13 11  8]
 [14 11  9  7]
 [12 10  8  6]
 [11  9  7  4]]
